# Transits of companions

## A notebook to make predictions of transit events for solved systems. The aims of this project are to search for transit timing variations and additioanl companions


### Imports

In [46]:
import bruce, numpy as np, matplotlib.pyplot as plt
from astropy.time import Time, TimeDelta
from astropy import units as u
from tqdm import tqdm
import numpy as np
import bruce, os
import matplotlib.pyplot as plt, gc
from itertools import combinations
from scipy.constants import G
import scipy.stats
from astropy.coordinates import SkyCoord, get_body
from astropy.time import Time, TimeDelta
from astropy.table import Table, Column, vstack
from astropy import units as u
from astroplan.utils import time_grid_from_range
from tqdm import tqdm 

from astroplan import Observer
from astropy.coordinates import  SkyCoord, get_body, get_sun
from astroquery.mast import Catalogs
from astropy.coordinates import SkyCoord, EarthLocation, AltAz




### Load NASA exoplanet archive

In [26]:
nasa = bruce.data.load_NASA_exoplanet_archive(unique_entries=True)
mask = (100*nasa['pl_radjerr1']/nasa['pl_radj']) < 20
mask &= ((100*nasa['pl_radjerr2']/nasa['pl_radj']) < 20)
mask &= ((100*nasa['pl_massjerr1']/nasa['pl_massj']) < 20)
mask &= ((100*nasa['pl_massjerr2']/nasa['pl_massj']) < 20)
mask &= (nasa['pl_orbper'] > 17)
mask &= (nasa['discoverymethod']=='Transit')

nasa = nasa[mask]

In [28]:
nasa['tic_id', 'pl_tranmid', 'pl_orbper']

tic_id,pl_tranmid,pl_orbper
str14,float64,float64
TIC 130162252,2455365.52723,95.272656
TIC 269701147,2459440.5491,28.579743
TIC 101011575,2459232.1682,18.87974
TIC 306263608,2458779.1919,52.563491
TIC 18310799,2457812.7177,17.30713
TIC 388804061,2457725.551189,32.940045
TIC 377780790,2455062.26648,45.294301
TIC 137686948,2455253.28756,1071.23205
TIC 399794329,--,29.323


In [62]:
def transit_plan(start=Time.now(), end = Time.now()+TimeDelta(360, format='jd'), resolution = 1*u.minute,
                tic_id=None, observatory='Paranal',
                 t_zero = None, period=None,width=None,
                sun_max_alt=-15, target_min_alt=30, moon_min_seperation=20,
                min_time_in_transit=None, min_frac_in_transit=None):
    # Now query tic8 
    tic8 = Catalogs.query_object('TIC{:}'.format(tic_id), radius=.02, catalog="TIC")[0]
    target = SkyCoord(tic8['ra']*u.deg, tic8['dec']*u.deg)

    time_range = time_grid_from_range((start,end), time_resolution=resolution)

    # Define observer's location (example: Mauna Kea, Hawaii)
    location = EarthLocation.of_site(observatory)  # Or use .from_geodetic(lon, lat, height)

    # Define the AltAz frame
    altaz_frame = AltAz(obstime=time_range, location=location)

    # Transform target coordinates to AltAz
    target_altaz = target.transform_to(altaz_frame)

    # Get Sun's ICRS coordinates
    sun_icrs = get_sun(time_range)

    # Transform the Sun's position to AltAz
    sun_altaz = sun_icrs.transform_to(altaz_frame)

    # Lets get the moon
    moon_icrs = get_body("moon", time_range, location)
    #moon_altaz = moon_icrs.transform_to(altaz_frame)

    # Moon-target_seperation
    moon_target_seperation = moon_icrs.separation(target)

    # Now get the time we can observe
    time_mask = (sun_altaz.alt.deg<sun_max_alt) & (target_altaz.alt.deg>target_min_alt) & (moon_target_seperation.deg>moon_min_seperation)
    #time_mask_sun = (sun_altaz.alt.deg<sun_max_alt) 

    # Now get the phases of all aliases
    phase = bruce.data.phase_times(time_range.jd, t_zero, period, phase_offset=0.2)
    phase_width = width / period

    # Now get a booleon mask of the transits
    visible_transits = np.abs(phase) < phase_width # Make for a nice plot
    visible_transits[~time_mask] = 0.

    if visible_transits.sum()==0:
        print('No transists visible')
        return Table()

    transit_nights = []
    # Now unpick this

    # Find unique nights of transits
    time_in_transit = time_range[visible_transits==1]
    segments = bruce.data.find_nights_from_data(time_in_transit.jd, dx_lim=0.2)

    # Get the parameters
    nights = [Observer(EarthLocation.of_site(observatory)).sun_set_time(time_in_transit[j][0], which='previous').datetime.__str__()[:10] for j in segments]
    for j in range(len(nights)):
        event = {}
        event['aliasP'] = 1
        event['aliasPer'] = period
        event['time_in_transit']=(time_in_transit[segments[j]].shape[0]*resolution).to(u.day).value
        event['frac_in_transit']=event['time_in_transit']/width
        event['moon_mean'] = np.mean(moon_target_seperation[visible_transits==1][segments[j]])
        event['night'] = nights[j]
        event['cycle_number'] = int(np.round((time_in_transit[segments[j]][0].jd - t_zero) / period))
        event['transit_center'] = t_zero + event['cycle_number']*period
        event['transit_width'] = width
        transit_nights.append([int(tic_id), str(observatory),float(sun_max_alt),float(target_min_alt),float(moon_min_seperation), str(event['night']) , int(event['aliasP']), float(event['aliasPer']), t_zero , float(event['time_in_transit']),float(event['frac_in_transit']), float(event['moon_mean'].value), int(event['cycle_number']), float(event['transit_center']), float(event['transit_width'])])


    t = Table()
    names=['tic_id','observatory','sun_max_alt','target_min_alt','moon_min_seperation','night','aliasP', 'aliasPer','alias_epoch', 'time_in_transit', 'frac_in_transit', 'moon_mean_sep', 'cycle_number', 'transit_center', 'transit_width']
    dtypes = [int,   str,         float,         float,         float,                  str,    int,       float,    float,          float,            float,            float,              int,          float,           float ]
    for i in range(len(names)):
        t.add_column(Column(np.array(transit_nights)[:,i].astype(dtypes[i]), name=names[i]))
    if min_time_in_transit is not None : t = t[np.array(t['time_in_transit'], dtype=np.float32) > min_time_in_transit]
    if min_frac_in_transit is not None : t = t[np.array(t['frac_in_transit'], dtype=np.float32) > min_frac_in_transit]
    return t



tables = []
for i in tqdm(range(len(nasa))[:]):
    try : 
        tables.append(transit_plan(start=Time.now(), end = Time.now()+TimeDelta(30, format='jd'), resolution = 1*u.minute,
                tic_id=int(nasa['tic_id'][i][4:]), observatory='Paranal',
                 t_zero = nasa['pl_tranmid'][i], period=nasa['pl_orbper'][i],width = nasa['pl_trandur'][i]/24.,
                sun_max_alt=-15, target_min_alt=30, moon_min_seperation=20,
                min_time_in_transit=None, min_frac_in_transit=None)) 
    except : pass








  0%|                                                                                                                                                                                                                                                                   | 0/32 [00:00<?, ?it/s]




  3%|███████▊                                                                                                                                                                                                                                                   | 1/32 [00:22<11:46, 22.80s/it]

No transists visible







  6%|███████████████▋                                                                                                                                                                                                                                           | 2/32 [00:44<11:11, 22.37s/it]

No transists visible







  9%|███████████████████████▌                                                                                                                                                                                                                                   | 3/32 [01:06<10:45, 22.24s/it]




 12%|███████████████████████████████▍                                                                                                                                                                                                                           | 4/32 [01:28<10:20, 22.15s/it]




 16%|███████████████████████████████████████▏                                                                                                                                                                                                                   | 5/32 [01:52<10:09, 22.57s/it]




 19%|███████████████████████████████████████████████                                                                   

...

No transists visible







 94%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▍               | 30/32 [10:52<00:44, 22.48s/it]

No transists visible







 97%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▏       | 31/32 [11:14<00:22, 22.31s/it]




100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 32/32 [11:36<00:00, 21.77s/it]

No transists visible


In [63]:
vstack(tables)


tic_id,observatory,sun_max_alt,target_min_alt,moon_min_seperation,night,aliasP,aliasPer,alias_epoch,time_in_transit,frac_in_transit,moon_mean_sep,cycle_number,transit_center,transit_width
int64,str32,float64,float64,float64,str32,int64,float64,float64,float64,float64,float64,int64,float64,float64
101011575,Paranal,-15.0,30.0,20.0,2025-11-07,1,18.87974,2459232.1682,0.03194444444444445,0.20890099909173482,60.7960136318979,93,2460987.9840200003,0.15291666666666667
101011575,Paranal,-15.0,30.0,20.0,2025-11-26,1,18.87974,2459232.1682,0.14583333333333334,0.9536784741144414,147.76174087160356,94,2461006.86376,0.15291666666666667
306263608,Paranal,-15.0,30.0,20.0,2025-11-06,1,52.563491,2458779.1919,0.17708333333333334,0.6484589563625267,32.046113839136204,42,2460986.858522,0.27308333333333334
18310799,Paranal,-15.0,30.0,20.0,2025-10-30,1,17.30713,2457812.7177,0.08819444444444445,0.6137044553976999,102.45064199640619,183,2460979.92249,0.14370833333333333
257527578,Paranal,-15.0,30.0,20.0,2025-10-30,1,54.18915,2458432.9798,0.18611111111111112,0.9816849816849818,73.53302558296588,47,2460979.86985,0.18958333333333333
149601126,Paranal,-15.0,30.0,20.0,2025-11-14,1,49.2519,2458335.4118,0.07430555555555556,0.32412456076578217,91.00666235837232,54,2460995.0143999998,0.22924999999999998
377064495,Paranal,-15.0,30.0,20.0,2025-11-26,1,25.62,2458521.882,0.03680555555555556,0.18213058419243988,171.22239747997676,97,2461007.0220000003,0.2020833333333333


In [69]:
nasa['pl_trandepstr']

--
--
--
--
--
--
--
"<div><span class=supersubNumber"">1.540</span><span class=""superscript"">+0.027</span><span class=""subscript"">-0.032</span></div>"""
--
1.62&plusmn;0.08
--
